In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten

In [2]:
df = pd.read_csv("train.txt", sep=";")

In [3]:
df.columns = ["Text", "Emotion"]

In [4]:
df.head()

,Text,Emotion
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [14]:
texts = df["Text"].tolist()
labels = df["Emotion"].tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [15]:
sequences = tokenizer.texts_to_sequences(texts)
max_len = max([len(seq) for seq in sequences])
pad_seq = pad_sequences(sequences, maxlen= max_len)

In [16]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels

array([5, 0, 3, ..., 2, 5, 4])

In [18]:
one_hot_label = keras.utils.to_categorical(labels)
one_hot_label

array([[0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [19]:
X_train , X_test, y_train, y_test = train_test_split(pad_seq, one_hot_label, test_size=0.2 )

In [26]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length= max_len))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_label[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/15
252/252 [==============================] - 11s 36ms/step - loss: 1.5439 - accuracy: 0.3919 - val_loss: 1.3168 - val_accuracy: 0.5172
Epoch 2/15
252/252 [==============================] - 10s 39ms/step - loss: 0.6722 - accuracy: 0.7842 - val_loss: 0.7445 - val_accuracy: 0.7404
Epoch 3/15
252/252 [==============================] - 10s 40ms/step - loss: 0.1057 - accuracy: 0.9782 - val_loss: 0.6982 - val_accuracy: 0.7693
Epoch 4/15
252/252 [==============================] - 9s 34ms/step - loss: 0.0331 - accuracy: 0.9943 - val_loss: 0.7011 - val_accuracy: 0.7782
Epoch 5/15
252/252 [==============================] - 10s 38ms/step - loss: 0.0172 - accuracy: 0.9970 - val_loss: 0.7318 - val_accuracy: 0.7742
Epoch 6/15
252/252 [==============================] - 10s 39ms/step - loss: 0.0138 - accuracy: 0.9975 - val_loss: 0.8546 - val_accuracy: 0.7524
Epoch 7/15
252/252 [==============================] - 12s 49ms/step - loss: 0.0112 - accuracy: 0.9975 - val_loss: 0.7572 - val_accuracy: 

In [28]:
input_text = "i am ever feeling nostalgic about the fireplace i will know that it is still on the property"

# Preprocess the input text"
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_len)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label[0])

1/1 [==============================] - 0s 197ms/step
love
